In [ ]:
#from urllib.request import urlretrieve
#import zipfile
#urlretrieve("http://www.models.life.ku.dk/sites/default/files/HPLCforweb_0.zip", "HPLCforweb_0.zip")
#zf = zipfile.ZipFile("./HPLCforweb_0.zip", "r")
#zf.extractall()
#zf.close()

In [ ]:
import pandas
import numpy
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from scipy.io import loadmat
olive = loadmat("olive.mat")
data = pandas.DataFrame(olive["HPLCforweb"][0][0][7])
data.index = olive["HPLCforweb"][0][0][12][0][0][0]
from sklearn.ensemble import IsolationForest
model = IsolationForest(contamination=0.05).fit(data)
data = data[model.predict(data) == 1]
data.T.plot(legend=None)
pyplot.show()

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, train_size=0.5, random_state=0)
print("training data")
print(train.index.value_counts())
print("\ntest data")
print(test.index.value_counts())

In [ ]:
# LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model = LinearDiscriminantAnalysis().fit(train.values, train.index)
print("model:", model)
print("score =", model.score(test.values, test.index))

In [ ]:
# SGD
from sklearn.linear_model import SGDClassifier
model = SGDClassifier().fit(train, train.index)
print("model:", model)
print("score =", model.score(test.values, test.index))

In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
grid = 20
p1 = numpy.logspace(-5, -4, num=grid)
parm = {"C": p1}
search = GridSearchCV(LogisticRegression(), parm).fit(train.values, train.index)
pyplot.scatter(p1, search.cv_results_["mean_test_score"])
pyplot.show()
model = search.best_estimator_.fit(train.values, train.index)
print("model:", model)
print("score =", model.score(test.values, test.index))

In [ ]:
# SVM
from sklearn.svm import SVC
grid = 20
p1 = numpy.logspace(0, 2, num=grid)
p2 = numpy.logspace(-3, -1, num=grid)
parm = {"C": p1, "gamma": p2}
search = GridSearchCV(SVC(kernel="rbf"), parm).fit(train.values, train.index)
map = pandas.DataFrame(search.cv_results_["mean_test_score"].reshape(grid, grid), index=p1, columns=p2)
import seaborn
seaborn.heatmap(map)
pyplot.ylabel("C")
pyplot.xlabel("gamma")
pyplot.show()
model = search.best_estimator_.fit(train.values, train.index)
print("model:", model)
print("score =", model.score(test.values, test.index))

In [ ]:
# Perceptron
from sklearn.linear_model import Perceptron
model = Perceptron().fit(train.values, train.index)
print("model:", model)
print("score =", model.score(test.values, test.index))

In [ ]:
# NN
from sklearn.neural_network import MLPClassifier
model = MLPClassifier().fit(train.values, train.index)
print("model:", model)
print("score =", model.score(test.values, test.index))

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier().fit(train, train.index)
print("model:", model)
print("score =", model.score(test.values, test.index))

In [ ]:
print(pandas.DataFrame(confusion_matrix(train.index, model.predict(train))))
print(classification_report(train.index, model.predict(train)))
print(pandas.DataFrame(confusion_matrix(test.index, model.predict(test))))
print(classification_report(test.index, model.predict(test)))